## Data Fetching Examples

### 1- Import the DataFetcher

In [12]:
from torchtrade.data_processors.ccxt_fetcher import CcxtFetcher
from torchtrade.envs.components.market import  Market
from torchtrade.envs.components.clock import Clock
from torchtrade.envs.components.trade import BoundedTrade
from datetime import datetime
import pandas as pd
import numpy as np

In [13]:
# Setting params
exchange = "binance"
symbols = ["BTC/USDT","ETH/USDT"]
timeFrame = "1d"

In [14]:
# Defining the market
datafetcher = CcxtFetcher(exchange,symbols,timeFrame)
df = datafetcher.fetch_data()


Fetching 1d timeframe data since 2017-08-17 00:00:00 until 2023-02-20 00:00:00


In [15]:
#Setting the clock
time_start = df.index[0][1]
time_end = df.index[-1][1]


clock = Clock(time_start,timeFrame)

mkt = Market(df)

#Market is observing the clock
clock.register_observer(mkt)


In [16]:
df

isTraded     open     high      low    close       volume
symbol   timestamp                                                           
BTC/USDT 2017-08-17        1  4261.48  4485.39  4200.74  4285.08   795.150377
         2017-08-18        1  4285.08  4371.52  3938.77  4108.37  1199.888264
         2017-08-19        1  4108.37  4184.69   3850.0  4139.98   381.309763
         2017-08-20        1  4120.98  4211.08  4032.62  4086.29   467.083022
         2017-08-21        1  4069.13  4119.62  3911.79   4016.0    691.74306
...                      ...      ...      ...      ...      ...          ...
ETH/USDT 2023-02-16        1  1674.92  1742.97  1633.72  1637.84  832723.6731
         2023-02-17        1  1637.83   1722.0  1630.51  1693.87   579934.663
         2023-02-18        1  1693.81  1713.13  1679.77  1691.62  267305.7813
         2023-02-19        1  1691.62  1726.59  1666.14  1679.75  401023.0477
         2023-02-20        1  1679.74  1699.54  1650.56  1692.85  170953.8091

[4028 rows x 6 columns]

Labeling

In [17]:
clock.reset()

stop_loss_levels = [1,2,3,4,5]
rr_levels = [1.5 , 2 , 2.5,3]

trades= []
while clock.timestamp < time_end : 
    data = mkt.data
    for index, row in data.iterrows():
        if row['isTraded'] == 1:
            trade = [[index[0],index[1],sl,rr] + [BoundedTrade(index[0],index[1],row['close'],sl,rr,mkt,clock.timeframe,0.9)] for sl in stop_loss_levels for rr in rr_levels] 
            trades+=trade
    clock.next()

trd = pd.DataFrame(trades, columns =['symbol', 'timestamp','stopLoss','riskReward','trade'])
trd.set_index(['symbol', 'timestamp', 'stopLoss', 'riskReward'], inplace=True)
trd["win/loss"] = trd['trade'].apply(lambda t : t.reward)
trd.to_csv('data.csv',index=False)
